In [ ]:
import requests
import pandas as pd
import numpy as np

def get_author_info(author_id, api_key):
    url = f'https://api.elsevier.com/content/search/author?co-author={author_id}'
    #url = f'https://api.elsevier.com/content/search/author?co-author=(57062645000)&apiKey=3334155c411aa69fa7f1328e0c50ad7c'
    headers = {'Accept': 'application/json', 'X-ELS-APIKey': api_key}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        print(data)
        print("Good response")
        return data
    else:
        print(f"Failed to retrieve author information. Status code: {response.status_code}")
        return None
    
def extract_coauthors_info(author_data):
    if 'search-results' in author_data:
        author_info = author_data['search-results']['opensearch:Query']
        coauthors_info = []
        if '@searchTerms' in author_info:
            documents = author_info['@searchTerms']
            print(documents)
            for document in documents:
                coauthors_info.append({'coauthor_id': document})
    
        return coauthors_info
    else:
        return None, []

def save_coauthors_to_csv(coauthors_info, output_file):
    data = []
    for coauthor in coauthors_info:
        data.append([coauthor])

    df = pd.DataFrame(data, columns=['Co-author IDs'])
    df.to_csv(output_file, index=False)

if __name__ == "__main__":
    api_key = '3334155c411aa69fa7f1328e0c50ad7c'
    author_id = '57062645000'
    output_file = 'coauthors_info.csv'
    author_data = get_author_info(author_id, api_key)
    if author_data:
        coauthors_info = extract_coauthors_info(author_data)

        if coauthors_info:
            save_coauthors_to_csv(coauthors_info, output_file)
            print(f"Co-authors' information saved to '{output_file}'")
        else:
            print("No co-authors found for the author.")
    else:
        print("Failed to retrieve author information.")
